---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 1 - Creating and Manipulating Graphs

Eight employees at a small company were asked to choose 3 movies that they would most enjoy watching for the upcoming company movie night. These choices are stored in the file `Employee_Movie_Choices.txt`.

A second file, `Employee_Relationships.txt`, has data on the relationships between different coworkers. 

The relationship score has value of `-100` (Enemies) to `+100` (Best Friends). A value of zero means the two employees haven't interacted or are indifferent.

Both files are tab delimited.

In [1]:
import networkx as nx
import pandas as pd
import numpy as np
from networkx.algorithms import bipartite


# This is the set of employees
employees = set(['Pablo',
                 'Lee',
                 'Georgia',
                 'Vincent',
                 'Andy',
                 'Frida',
                 'Joan',
                 'Claude'])

# This is the set of movies
movies = set(['The Shawshank Redemption',
              'Forrest Gump',
              'The Matrix',
              'Anaconda',
              'The Social Network',
              'The Godfather',
              'Monty Python and the Holy Grail',
              'Snakes on a Plane',
              'Kung Fu Panda',
              'The Dark Knight',
              'Mean Girls'])


# you can use the following function to plot graphs
# make sure to comment it out before submitting to the autograder
def plot_graph(G, weight_name=None):
    '''
    G: a networkx G
    weight_name: name of the attribute for plotting edge weights (if G is weighted)
    '''
    %matplotlib notebook
    import matplotlib.pyplot as plt
    
    plt.figure()
    pos = nx.spring_layout(G)
    edges = G.edges()
    weights = None
    
    if weight_name:
        weights = [int(G[u][v][weight_name]) for u,v in edges]
        labels = nx.get_edge_attributes(G,weight_name)
        nx.draw_networkx_edge_labels(G,pos,edge_labels=labels)
        nx.draw_networkx(G, pos, edges=edges, width=weights);
    else:
        nx.draw_networkx(G, pos, edges=edges);

### Question 1

Using NetworkX, load in the bipartite graph from `Employee_Movie_Choices.txt` and return that graph.

*This function should return a networkx graph with 19 nodes and 24 edges*

In [2]:
def answer_one():
        
    # Creates bipartite graph
    G = nx.Graph()
    
    # Adding Nodes
    G.add_nodes_from(employees,bipartite=0)
    G.add_nodes_from(movies,bipartite=1)

    # Loading Employees to Movies edges
    bigraph_df  = pd.read_csv('Employee_Movie_Choices.txt',sep='\t',names = ['Employees','Movies'],header=0)
    # *Since: 
    # # emp2mov = nx.from_pandas_dataframe(bigraph_df,'Employees','Movies')
    # # edges = emp2mov.edges() 
    # *does not work, we use:
    emp2mov_edges = list(zip(bigraph_df['Employees'],bigraph_df['Movies']))
    
    # Creating the edges between nodes in each side
    G.add_edges_from(emp2mov_edges)
    
#     #Plotting the graph
#     plot_graph(G)
    
    return G

In [3]:
# #employees
# B = answer_one()
# print('Nodes: {} ; Edges: {}'.format(len(B.nodes()),len(B.edges())) )

### Question 2

Using the graph from the previous question, add nodes attributes named `'type'` where movies have the value `'movie'` and employees have the value `'employee'` and return that graph.

*This function should return a networkx graph with node attributes `{'type': 'movie'}` or `{'type': 'employee'}`*

In [4]:
def answer_two():
    
    B = answer_one()
    B.add_nodes_from(employees,type='employee')
    B.add_nodes_from(movies,type='movie')
    return B

In [5]:
# B = answer_two()
# B.nodes(data=True)

### Question 3

Find a weighted projection of the graph from `answer_two` which tells us how many movies different pairs of employees have in common.

*This function should return a weighted projected graph.*

In [6]:
def answer_three():
        
    B = answer_two()
    b_weight_proj = bipartite.weighted_projected_graph(B,employees)
    
    #plot_graph(b_weight_proj,'weight')
    
    return  b_weight_proj

In [7]:
# answer_three()

### Question 4

Suppose you'd like to find out if people that have a high relationship score also like the same types of movies.

Find the Pearson correlation ( using `DataFrame.corr()` ) between employee relationship scores and the number of movies they have in common. If two employees have no movies in common it should be treated as a 0, not a missing value, and should be included in the correlation calculation.

*This function should return a float.*

In [8]:
def answer_four():
        
    # Retrieving weighted projected graph
    b_weighted_proj = answer_three()
    # Converting edges and weights con Dataframe
    ## No. Movies in common for each employee
    mov_common_df = nx.to_pandas_dataframe(b_weighted_proj)
    
    # Retrieving relationship score as Dataframe
    ## Relationship score
    rlts_score_df = pd.read_csv('Employee_Relationships.txt',sep='\t',names = ['Employee1','Employee2','Score'],index_col=False)
    
    # Merging both in one Dataframe
    corr_df = rlts_score_df.copy()
    corr_df['mov_common']= corr_df.apply(lambda mc: mov_common_df.loc[mc['Employee1'],mc['Employee2']],axis = 1) #Preserve order given in relationship dataframe
                                                                                                                 # and looking up to movies in common Dataframe
    corr_score = corr_df['Score'].corr(corr_df['mov_common'])
    
    return corr_score

In [9]:
#answer_four()